In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive/')
path = "./gdrive/My Drive/Colab Notebooks/金融大数据"
os.chdir(path)


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import pandas as pd
import jieba
import re
from collections import Counter

jieba.load_userdict('user_dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.760 seconds.
Prefix dict has been built successfully.


## 数据分析

### 加载数据集

In [3]:
data_raw = pd.read_csv("raw_data/股评数据.csv",sep = '\t', header = None, encoding="gbk")

In [4]:
print(data_raw.head())

                                                   0
0  9   走强需要时日，但是，价值明摆着呢。以下为问答私密内容，你已经解锁获得查看权限T+0，...
1  9   详细回答见私密部分。以下为问答私密内容，你已经解锁获得查看权限该股具有非常强的代表性...
2  1   详细回答见私密部分。以下为问答私密内容，你已经解锁获得查看权限新疆板块龙头品种，还可...
3  1   详细回答见私密部分。以下为问答私密内容，你已经解锁获得查看权限强势震荡,,,,,,,...
4  0   详细回答见私密部分。以下为问答私密内容，你已经解锁获得查看权限连续大涨之后出现一定的...


### 设置停用词
第一个停用词范围比较广，给聚类用；第二个只删除了一些废话比如“您已解锁....”和标点符号、数字，连贯性比较好，给bert用。

In [5]:
# 根据stopwords.txt创建一个停用词表

stopwords1 = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
stopwords2 = [line.strip() for line in open('stopwords2.txt',encoding='UTF-8').readlines()]

In [6]:
data_raw.loc[5][0]

'1   详细回答见私密部分。以下为问答私密内容，你已经解锁获得查看权限算是热门板块之一，还不错！,,,,,,,,,,,,,,,,,,,,,,,'

In [7]:
def split(text):
    #害，我让用'\t'分割,最后给我的是仨空格分割的
    lst = text.strip().split('   ')
    return lst

清洗数据，包括去掉停用词、删掉数字、分词

In [8]:
def cleaner(sentence, idx = 1):
    if idx == 1:
        stopwords = stopwords1
    else:
        stopwords = stopwords2
    s = re.sub("\d+","",sentence)
    words = jieba.cut(s)
    outStr = ''
    for word in words:
        if word not in stopwords:
            outStr += word
            outStr += ' '
    return outStr[:-1]


In [9]:
def data_proccessing(df):
    res_x = []
    res_y = []
    res_for_bert = []
    for index, row in df.iterrows():
        try:
            y,x = split(row[0])
            if(y == '0' or y == '1' or y == '9'):
                x1 = cleaner(x)
                res_y.append(y)
                res_x.append(x1)
                res_for_bert.append(cleaner(x, 2))
        except:
            continue
    return res_x, res_for_bert, res_y

X1用于聚类，X2用于训练bert

In [10]:
X1,X2,Y = data_proccessing(data_raw)

In [11]:
print(len(X1),len(Y))

19978 19978


In [12]:
print(X1[0:5])
print(X2[0:5])

['走强 时日 价值 明摆着 T 期权 注册 制 题材 兑现 至少 涨停', '该股 强 代表性 加速 上涨 涨停 持股', '新疆 板块 龙头 品种 持有', '强势 震荡', '连续 大涨 下跌 属 反复 区域 低点']
['走强 需要 时日 但是 价值 明摆着 呢 T 期权 注册 制 每个 题材 兑现 都 至少 一个 涨停', '该股 具有 非常 强 的 代表性 正在 加速 上涨 涨停 后 继续 持股', '新疆 板块 龙头 品种 还 可 继续 持有 一下', '强势 震荡', '连续 大涨 之后 出现 一定 的 下跌 属 正常 还有 反复 不过 目前 区域 绝不 是 低点']


### 样本分析

In [13]:
cls = list(set(Y))

In [14]:
print(cls)

['0', '1', '9']


In [15]:
Counter(Y)

Counter({'0': 1825, '1': 13761, '9': 4392})

In [16]:
X_cls1 = []
X_o = []
Y_o = []
Xb_cls1 = []
Xb_o = []
for index, i in enumerate(Y):
    if(i == '1'):
        X_cls1.append(X1[index])
        Xb_cls1.append(X2[index])
    else:
        X_o.append(X1[index])
        Xb_o.append(X2[index])
        Y_o.append(i)

In [17]:
print(len(X_cls1),len(Y_o))

13761 6217


In [18]:
print(X_cls1[0])
print(Xb_cls1[0])

新疆 板块 龙头 品种 持有
新疆 板块 龙头 品种 还 可 继续 持有 一下


把"1"聚成3类比较合适

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import Birch,KMeans
from sklearn.manifold import TSNE


In [20]:
tv = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None)

In [21]:
train_texts = X_cls1

In [22]:
tv_fit = tv.fit_transform(train_texts)

In [23]:
tv.get_feature_names()

['abc',
 'admin',
 'ae',
 'af',
 'all',
 'and',
 'apec',
 'a股',
 'bdi',
 'boll',
 'bu',
 'cn',
 'com',
 'ctrl',
 'da',
 'de',
 'detail',
 'dif',
 'dq',
 'eprint',
 'eps',
 'erp',
 'etf',
 'facebook',
 'f盘',
 'gdp',
 'gqy',
 'home',
 'homos',
 'http',
 'h股',
 'ic',
 'id',
 'ie',
 'if',
 'in',
 'ind',
 'iphone',
 'ipo',
 'it',
 'jjj',
 'kdj',
 'led',
 'licaishi',
 'ma',
 'macd',
 'msci',
 'mx',
 'no',
 'ok',
 'oo',
 'opcon',
 'or',
 'oto',
 'outlets',
 'pb',
 'pcr',
 'pe',
 'plan',
 'ppp',
 'ps',
 'pta',
 'px',
 'qdii',
 'qfii',
 'qq',
 'rm',
 'rqfii',
 'rt',
 'sdr',
 'sh',
 'shang',
 'sina',
 'st',
 'sz',
 'ta',
 'taozheba',
 'tcl',
 'tct',
 'thx',
 'ti',
 'topnav',
 'trf',
 'view',
 'vip',
 'vr',
 'weibo',
 'wen',
 'wto',
 'wvr',
 'xx',
 'ylgf',
 'yoy',
 '一万',
 '一万五',
 '一万倍',
 '一上',
 '一上午',
 '一下一下',
 '一下周',
 '一下子',
 '一不小心',
 '一两个',
 '一两只',
 '一两周',
 '一两天',
 '一两年',
 '一两日',
 '一两根',
 '一两次',
 '一个千',
 '一个双',
 '一个多月',
 '一个头',
 '一个月',
 '一个点',
 '一举',
 '一买',
 '一买股',
 '一事',
 '一人',
 '一份',
 '一会',
 

In [24]:
tv_fit.toarray().shape

(13761, 13412)

In [25]:
weight = tv_fit.toarray()

In [26]:
print(weight[0])

[0. 0. 0. ... 0. 0. 0.]


In [27]:
def matrixPCA(weight,dimension):
    pca = PCA(n_components = dimension)  # 初始化PCA
    pcaMatrix = pca.fit_transform(weight)        # 返回降维后的数据
    print("降维之前的权重维度：",weight.shape)
    print("降维之后的权重维度：",pcaMatrix.shape)
    return pcaMatrix

def birch(matrix,k):
    clusterer = Birch(n_clusters=k)  # 分成簇的个数
    y = clusterer.fit_predict(matrix)    # 聚类结果
    return y

def kmeans(matrix,k):
    clusterer = KMeans(n_clusters=k)  # 分成簇的个数
    y = clusterer.fit_predict(matrix)
    return y

In [28]:
weightPCA = weight
weightPCA = matrixPCA(weight, dimension = 512) 

降维之前的权重维度： (13761, 13412)
降维之后的权重维度： (13761, 512)


In [29]:
print(weightPCA[0])

[-1.27434849e+00 -1.13420566e+00  2.84981197e-01  3.42630524e-01
 -9.25764098e-01 -1.60719768e-01  4.35836330e-01  1.05837526e-01
  8.68074093e-01  9.11201192e-01  1.15329485e+00 -9.16891752e-01
  1.11107275e+00  6.54904865e-01  3.08053611e-01 -2.71830159e-01
 -4.68442604e-01  8.14944529e-01 -3.61336211e-01  8.91889998e-01
  6.12513035e-01 -6.70922255e-01  3.96063184e-01  2.81382109e-01
  4.95584878e-01  9.50510307e-01 -4.64765644e-01  2.23430889e-01
 -5.45293409e-01 -3.28512859e-01 -3.66242094e-02 -8.48986843e-02
  1.88636504e-01 -2.03895511e-01 -7.37142017e-01 -1.08852151e+00
 -6.19161987e-01 -2.00650412e-01 -3.88990217e-01  8.27872397e-01
  2.26167280e-01 -9.83112974e-01  1.35044911e-01  1.15292587e-02
 -5.29821500e-01 -6.63217203e-01 -1.34164518e-01  7.46760163e-01
  1.17855925e+00 -2.12175990e-01 -8.79652307e-01 -3.27961645e-01
  1.58663796e-02 -5.48323312e-01  5.49722443e-01  1.80065612e-01
 -9.31140098e-01  7.35890077e-01 -5.32229424e-01  4.55673481e-01
 -1.31417811e-01 -1.12626

In [30]:
tsne = TSNE(n_components=3)
t_data = tsne.fit_transform(weightPCA)

In [31]:
print(t_data.shape)

(13761, 3)


In [32]:
k = 4

In [33]:
y_ = kmeans(t_data,k)

In [34]:
Counter(y_)

Counter({0: 3237, 1: 3326, 2: 3774, 3: 3424})

In [35]:
print(y_[0:10])

[1 2 1 3 1 1 1 3 3 1]


In [36]:
last_X = []
last_Xb = []
last_Y = []
for index, i in enumerate(y_):
  last_X.append(X_cls1[index])
  last_Xb.append(Xb_cls1[index])
  if i == 0:
    last_Y.append('1')
  elif i == 1:
    last_Y.append('2')
  elif i == 2:
    last_Y.append('3')
  else:
    last_Y.append('4')


In [37]:
for index, i in enumerate(Y_o):
  last_X.append(X_o[index])
  last_Xb.append(Xb_o[index])
  if i == '0':
    last_Y.append('0')
  elif i == '9':
    last_Y.append('5')

In [38]:
print(len(last_X),len(last_Y))
print(last_X[0:5])
print(last_Xb[0:5])
print(last_Y[0:5])

19978 19978
['新疆 板块 龙头 品种 持有', '强势 震荡', '算是 热门 板块 不错', '去年 妖股 突破 日 均线 短线 空间 打卡', '反弹 建议 先 出货']
['新疆 板块 龙头 品种 还 可 继续 持有 一下', '强势 震荡', '算是 热门 板块 之一 还 不错', '是 去年 的 要 妖股 之一 如果 突破 日 均线 短线 空间 将 被 打卡', '反弹 后 建议 先 出货']
['2', '3', '2', '4', '2']


In [39]:
cnt = Counter(last_Y)
print(cnt)

Counter({'5': 4392, '3': 3774, '4': 3424, '2': 3326, '1': 3237, '0': 1825})


In [40]:
my_dict = {0:last_Y,1:last_Xb,2:last_X}
new_df = pd.DataFrame(my_dict)

In [41]:
print(new_df.head())

   0                                       1                       2
0  2                新疆 板块 龙头 品种 还 可 继续 持有 一下          新疆 板块 龙头 品种 持有
1  3                                   强势 震荡                   强势 震荡
2  2                        算是 热门 板块 之一 还 不错             算是 热门 板块 不错
3  4  是 去年 的 要 妖股 之一 如果 突破 日 均线 短线 空间 将 被 打卡  去年 妖股 突破 日 均线 短线 空间 打卡
4  2                            反弹 后 建议 先 出货              反弹 建议 先 出货


In [42]:
new_df.to_csv("data2.csv",sep = "\t",index=None)